In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!mkdir data

In [0]:
!cp -r "drive/My Drive/BirdsSegm/public_tests/00_test_val_input/train" data

In [0]:
import os

root_dir   = '/content/drive/My Drive/BirdsSegm/'
train_dir  = os.path.join('data', 'train')
images_dir = os.path.join(train_dir, 'images')
masks_path = os.path.join(train_dir, 'gt')
model_dir  = os.path.join(root_dir, 'models')

In [0]:
from skimage.io import imread
from skimage.transform import resize
from skimage import img_as_float
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Input, concatenate, UpSampling2D
from keras.optimizers import Adam
from keras.initializers import he_normal
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import load_model

In [0]:
def unet(input_size = (256,256,3), seed=42, pretrained_weights = None, base = 32, actv='relu'):
    inputs = Input(input_size)
    kern_init = kernel_initializer=he_normal(seed=seed)
    conv1 = Conv2D(base,    3, activation=actv, padding='same', )(inputs)
    conv1 = Conv2D(base,    3, activation=actv, padding='same', kernel_initializer=kern_init)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(base*2,  3, activation=actv, padding='same', kernel_initializer=kern_init)(pool1)
    conv2 = Conv2D(base*2,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(base*4,  3, activation=actv, padding='same', kernel_initializer=kern_init)(pool2)
    conv3 = Conv2D(base*4,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(base*8,  3, activation=actv, padding='same', kernel_initializer=kern_init)(pool3)
    conv4 = Conv2D(base*8,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(base*16, 3, activation=actv, padding='same', kernel_initializer=kern_init)(pool4)
    conv5 = Conv2D(base*16, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6   = Conv2D(base*8,  2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(base*8,  3, activation=actv, padding='same', kernel_initializer=kern_init)(merge6)
    conv6 = Conv2D(base*8,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv6)

    up7   = Conv2D(base*4,  2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(base*4,  3, activation=actv, padding='same', kernel_initializer=kern_init)(merge7)
    conv7 = Conv2D(base*4,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv7)

    up8   = Conv2D(base*2,  2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(base*2,  3, activation=actv, padding='same', kernel_initializer=kern_init)(merge8)
    conv8 = Conv2D(base*2,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv8)

    up9   = Conv2D(base,    2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(base,    3, activation=actv, padding='same', kernel_initializer=kern_init)(merge9)
    conv9 = Conv2D(base,    3, activation=actv, padding='same', kernel_initializer=kern_init)(conv9)
    conv9 = Conv2D(2,       3, activation=actv, padding='same', kernel_initializer=kern_init)(conv9)
    conv10 = Conv2D(1,      1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy', iou])
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model


def unet_vgg16(input_size = (256,256,3), seed=42, pretrained_weights = None, actv='relu'):
    inputs = Input(input_size)
    kern_init = kernel_initializer=he_normal(seed=seed)
    conv1 = Conv2D(64,  3, activation=actv, padding='same', )(inputs)
    conv1 = Conv2D(64,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation=actv, padding='same', kernel_initializer=kern_init)(pool1)
    conv2 = Conv2D(128, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation=actv, padding='same', kernel_initializer=kern_init)(pool2)
    conv3 = Conv2D(256, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv3)
    conv3 = Conv2D(256, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(pool3)
    conv4 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv4)
    conv4 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(pool4)
    conv5 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv5)
    conv5 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6   = Conv2D(512, 2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(merge6)
    conv6 = Conv2D(512, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv6)

    up7   = Conv2D(256, 2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation=actv, padding='same', kernel_initializer=kern_init)(merge7)
    conv7 = Conv2D(256, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv7)

    up8   = Conv2D(128, 2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation=actv, padding='same', kernel_initializer=kern_init)(merge8)
    conv8 = Conv2D(128, 3, activation=actv, padding='same', kernel_initializer=kern_init)(conv8)

    up9   = Conv2D(64,  2, activation=actv, padding='same', kernel_initializer=kern_init)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64,  3, activation=actv, padding='same', kernel_initializer=kern_init)(merge9)
    conv9 = Conv2D(64,  3, activation=actv, padding='same', kernel_initializer=kern_init)(conv9)
    conv9 = Conv2D(2,   3, activation=actv, padding='same', kernel_initializer=kern_init)(conv9)
    conv10 = Conv2D(1,  1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy', iou])
    
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    for i in range(len(vgg_conv.layers)):
        weights = vgg_conv.layers[i].get_weights()
        layer = model.layers[i]
        layer.set_weights(weights)
        layer.trainable = False

    return model


def iou(y_true, y_pred):
    smooth=1e-5
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


def trainGenerator(
    batch_size,
    image_path,
    mask_path,
    aug_dict,
    image_color_mode = "rgb",
    mask_color_mode = "grayscale",
    target_size = (256,256),
    seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(
        rescale=1./255,
        samplewise_center=True,
        samplewise_std_normalization=True,
        **aug_dict)
    mask_datagen = ImageDataGenerator(rescale=1./255, **aug_dict)
    image_generator = image_datagen.flow_from_directory(
        image_path,
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        interpolation='bilinear',
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        mask_path,
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        interpolation='bilinear',
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        yield (img,mask)

In [0]:
target_size=(256,256)

data_gen_args = dict(
    rotation_range=0.3,
    width_shift_range=0.15,
    height_shift_range=0.15,
    #shear_range=0.05,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

batch_size=16

train_gen = trainGenerator(
    batch_size=batch_size,
    image_path=images_dir,
    mask_path=masks_path,
    aug_dict=data_gen_args,
    target_size=target_size)

val_gen = trainGenerator(
    batch_size=batch_size,
    image_path=os.path.join(root_dir, 'public_tests', '00_test_val_input', 'test', 'images'),
    mask_path=os.path.join(root_dir, 'public_tests', '00_test_val_gt'),
    aug_dict={},
    target_size=target_size)

from tensorflow.keras.models import load_model
model_path = os.path.join(model_dir, 'model10.hdf5')
model = unet(input_size=(None, None, 3), base=64)
#model = load_model(model_path, custom_objects={'iou': iou})
model_checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True)
model.fit_generator(
    train_gen,
    steps_per_epoch=8382 // batch_size,
    #validation_data=val_gen,
    #validation_steps=20,
    epochs=30,
    callbacks=[model_checkpoint])
model.save(model_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/30
Found 8382 images belonging to 200 classes.
Found 8382 images belonging to 200 classes.
523/523 [==============================] - 333s 636ms/step - loss: 0.2557 - acc: 0.8442 - iou: 0.2371

Epoch 00001: loss improved from inf to 0.25573, saving model to /content/drive/My Drive/BirdsSegm/models/model10.hdf5
Epoch 2/30
523/523 [==============================] - 326s 623ms/step - loss: 0.2027 - acc: 0.8748 - iou: 0.2947

Epoch 00002: loss improved from 0.25573 to 0.20272, saving model to /content/drive/My Drive/BirdsSegm/models/model10.hdf5
Epoch 3/30
523/523 [==============================] - 327s 625ms/step - loss: 0.1865 - acc: 0.8840 - iou: 0.3208

Epoch 00003: loss improved from 0.20272 to 0.18653, saving model to /content/drive/My Drive/BirdsSegm/models/model10.hdf5
Epoch 4/30
523/523 [==============================] - 328s 627ms/step - loss: 0.1741 - acc: 0.8895 - iou: 0.3435

Epoch 00004: loss improved from 0.18653 to 0.17411, saving model to /content/drive/My Drive/Bi

In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy')
model.save(os.path.join(model_dir, 'model10_1.hdf5'))

In [0]:
model.fit_generator(
    train_gen,
    steps_per_epoch=8382 // batch_size,
    #validation_data=val_gen,
    #validation_steps=20,
    epochs=10,A
    callbacks=[model_checkpoint])

In [0]:
target_size=(256,256)

data_gen_args = dict(
    rotation_range=0.3,
    width_shift_range=0.15,
    height_shift_range=0.15,
    #shear_range=0.05,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

batch_size=16

train_gen = trainGenerator(
    batch_size=batch_size,
    image_path=images_dir,
    mask_path=masks_path,
    aug_dict=data_gen_args,
    target_size=target_size)

val_gen = trainGenerator(
    batch_size=batch_size,
    image_path=os.path.join('public_tests', 'public_tests', '00_test_val_input', 'test', 'images'),
    mask_path=os.path.join('public_tests', 'public_tests', '00_test_val_gt'),
    aug_dict={},
    target_size=target_size)

model_path = os.path.join(model_dir, 'model_vgg16.hdf5')
model = unet_vgg16(input_size=(None, None, 3))
#model = load_model(model_path, custom_objects={'iou': iou})
model_checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True)
model.fit_generator(
    train_gen,
    steps_per_epoch=8382 // batch_size,
    #validation_data=val_gen,
    #validation_steps=40,
    epochs=40,
    callbacks=[model_checkpoint])
model.save(model_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/40
Found 8382 images belonging to 200 classes.
Found 8382 images belonging to 200 classes.
523/523 [==============================] - 330s 630ms/step - loss: 0.1606 - acc: 0.8973 - iou: 0.3676

Epoch 00001: loss improved from inf to 0.16063, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16.hdf5
Epoch 2/40
523/523 [==============================] - 320s 612ms/step - loss: 0.1366 - acc: 0.9107 - iou: 0.4059

Epoch 00002: loss improved from 0.16063 to 0.13662, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16.hdf5
Epoch 3/40
523/523 [==============================] - 325s 621ms/step - loss: 0.1309 - acc: 0.9123 - iou: 0.4208

Epoch 00003: loss improved from 0.13662 to 0.13094, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16.hdf5
Epoch 4/40
523/523 [==============================] - 325s 621ms/step - loss: 0.1255 - acc: 0.9134 - iou: 0.4353

Epoch 00004: loss improved from 0.13094 to 0.12553, saving model to /content/drive

In [15]:
target_size=(256,256)

data_gen_args = dict(
    rotation_range=0.3,
    width_shift_range=0.15,
    height_shift_range=0.15,
    #shear_range=0.05,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

batch_size=16

train_gen = trainGenerator(
    batch_size=batch_size,
    image_path=images_dir,
    mask_path=masks_path,
    aug_dict=data_gen_args,
    target_size=target_size)

model_path = os.path.join(model_dir, 'model_vgg16_1.hdf5')
model = load_model(os.path.join(model_dir, 'model_vgg16.hdf5'), custom_objects={'iou': iou})
model_checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True)
model.compile(optimizer = Adam(lr = 4e-5), loss = 'binary_crossentropy', metrics = ['accuracy', iou])
model.fit_generator(
    train_gen,
    steps_per_epoch=8382 // batch_size,
    epochs=30,
    callbacks=[model_checkpoint])
model.save(model_path)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Epoch 1/30
Found 8382 images belonging to 200 classes.
Found 8382 images belonging to 200 classes.
523/523 [==============================] - 298s 569ms/step - loss: 0.0642 - acc: 0.9183 - iou: 0.7033

Epoch 00001: loss improved from inf to 0.06417, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16_1.hdf5
Epoch 2/30
523/523 [==============================] - 287s 549ms/step - loss: 0.0628 - acc: 0.9187 - iou: 0.7079

Epoch 00002: loss improved from 0.06417 to 0.06280, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16_1.hdf5
Epoch 3/30
523/523 [==============================] - 289s 552ms/step - loss: 0.0625 - acc: 0.9187 - iou: 0.7094

Epoch 00003: loss improved from 0.06280 to 0.06254, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16_1.hdf5
Epoch 4/30
523/523 [==============================] - 291s 557ms/step - loss: 0.0622 - acc: 0.9185 - iou: 0.7114

Epoch 00004: loss improved from 0.06254 to 0.06217, saving model to /content

In [16]:
for layer in model.layers:
    layer.trainable = True
model.compile(optimizer = Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy', iou])
model_path = os.path.join(model_dir, 'model_vgg16_2.hdf5')
model_checkpoint = ModelCheckpoint(model_path, monitor='loss', verbose=1, save_best_only=True)
model.fit_generator(
    train_gen,
    steps_per_epoch=8382 // batch_size,
    epochs=40,
    callbacks=[model_checkpoint])

Epoch 1/40
523/523 [==============================] - 311s 595ms/step - loss: 0.0543 - acc: 0.9194 - iou: 0.7524

Epoch 00001: loss improved from inf to 0.05427, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16_2.hdf5
Epoch 2/40
523/523 [==============================] - 304s 582ms/step - loss: 0.0543 - acc: 0.9191 - iou: 0.7522

Epoch 00002: loss did not improve from 0.05427
Epoch 3/40
523/523 [==============================] - 304s 582ms/step - loss: 0.0541 - acc: 0.9191 - iou: 0.7530

Epoch 00003: loss improved from 0.05427 to 0.05408, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16_2.hdf5
Epoch 4/40
523/523 [==============================] - 317s 605ms/step - loss: 0.0539 - acc: 0.9191 - iou: 0.7534

Epoch 00004: loss improved from 0.05408 to 0.05387, saving model to /content/drive/My Drive/BirdsSegm/models/model_vgg16_2.hdf5
Epoch 5/40
523/523 [==============================] - 320s 612ms/step - loss: 0.0539 - acc: 0.9190 - iou: 0.7543

Epo